In [15]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [16]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(list_hh, i))
    ides.append(preprocessed_ide(list_hh, i))
    if i < 5:
        dialogues.append(preprocessed_dialogue(list_woz, i))
        ides.append(preprocessed_ide(list_woz, i))


9
5
9
5
9
5
9
5
9
5
9
9
9
9


In [17]:
dialogue_input, dialogue_output = [], []

def process_data(data):
    input, output = [], []
    for i in data:
        s1, s2 = data[0], data[1]
#         print(s1)
        for i in range(max(len(s1), len(s2))):
            if i > 1:
                temp = []
                temp.append(s1[i-1])
                temp.append(s2[i-1])
                padding(temp[0])
                padding(temp[1])
                input.append(temp)
                output.append(s1[i][0])
#                 print(temp)
                
                if i < len(s2):
                    temp = []
                    temp.append(s2[i-1])
                    temp.append(s1[i])
                    padding(temp[0])
                    padding(temp[1])
                    input.append(temp)
                    output.append(s2[i][0])
    return input, output
    
def padding(data):
    while len(data) < 21: data.append(0)
        
data = []
for i in dialogues:
    input, output = process_data(i)
    dialogue_input.extend(input)
    dialogue_output.extend(output)
# print(len(input), len(output))


In [18]:
from keras import backend as K


def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=10):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
#         return the score for more statistics-in the future reference
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])
        print(len(X_test), len(y_test))
    print(scores)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = np.array(dialogue_input)
y = np.array(dialogue_output)

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 


18/18 - 1s - loss: 67.9514 - accuracy: 0.2166 - 501ms/epoch - 28ms/step
554 554
18/18 - 0s - loss: 66.3951 - accuracy: 0.2310 - 86ms/epoch - 5ms/step
554 554
18/18 - 0s - loss: 71.1531 - accuracy: 0.2604 - 114ms/epoch - 6ms/step
553 553
18/18 - 0s - loss: 72.0502 - accuracy: 0.2387 - 93ms/epoch - 5ms/step
553 553
18/18 - 0s - loss: 66.8777 - accuracy: 0.2550 - 106ms/epoch - 6ms/step
553 553
18/18 - 0s - loss: 69.1256 - accuracy: 0.2568 - 104ms/epoch - 6ms/step
553 553
18/18 - 0s - loss: 63.7420 - accuracy: 0.2586 - 97ms/epoch - 5ms/step
553 553
18/18 - 0s - loss: 72.6072 - accuracy: 0.2369 - 113ms/epoch - 6ms/step
553 553
18/18 - 0s - loss: 61.7995 - accuracy: 0.2532 - 96ms/epoch - 5ms/step
553 553
18/18 - 0s - loss: 62.2963 - accuracy: 0.2586 - 104ms/epoch - 6ms/step
553 553
[0.2166064977645874, 0.23104692995548248, 0.2603978216648102, 0.23869800567626953, 0.2549728751182556, 0.2567811906337738, 0.258589506149292, 0.23688969016075134, 0.2531645596027374, 0.258589506149292]
Cross-valid